In [1]:
##### IMPORTACION DE LAS LIBRERIAS PARA LA LIMPIEZA Y TABULACION DE DATOS #####
import pandas as pd
import tabula
import os
import xlsxwriter

# Ruta de la carpeta donde están los archivos PDF
carpeta_pdf = r'D:\DOCUMENTOS\Resumenes bancarios\e-resumen sin clave'
# Ruta de la carpeta donde se guardarán los archivos Excel generados
carpeta_guardar = r'D:\DOCUMENTOS\Resumenes Bancarios en Excel'

In [2]:
# Nombres de las columnas deseadas
columnas = ["Fecha", "Descripcion", "Referencia", "Debitos", "Creditos", "Saldos"]

# Función para procesar cada archivo PDF
def procesar_pdf(ruta_pdf):
    # Leer el contenido del PDF utilizando tabula
    tablas = tabula.read_pdf(ruta_pdf, pages='all',
                             relative_area=True, 
                             relative_columns=True, 
                             area=[26, 0, 86, 100], 
                             columns=[11, 43, 55, 68, 84], silent=True)

    # Renombrar las columnas de cada DataFrame en la lista
    dfs_renombrados = [df.rename(columns=dict(zip(df.columns, columnas))) for df in tablas]

    # Unir todos los DataFrames en uno solo
    df_unido = pd.concat(dfs_renombrados, ignore_index=True)

    # Función para obtener el número de cuenta desde la columna "Descripcion"
    def obtener_cuenta(row):
        if pd.notna(row['Descripcion']) and "NRO.:" in row['Descripcion']:
            cuenta_texto = row['Descripcion'].split(":")[-1].strip()
            if pd.notna(row['Referencia']):
                cuenta_texto += row['Referencia'].replace(" ", "")
            return cuenta_texto
        else:
            return None

    # Aplicar la columna 'Cuenta'
    df_unido["Cuenta"] = df_unido.apply(obtener_cuenta, axis=1)
    df_unido['Cuenta'] = df_unido['Cuenta'].ffill()

    # Filtrar por las columnas 'Fecha' y 'Descripcion'
    df_unido = df_unido.loc[df_unido['Fecha'].str.contains(r'^[^/]*\/[^/]*\/[^/]*$', na=False) | 
                            df_unido['Descripcion'].str.contains('SALDO ULTIMO EXTRACTO', case=False, na=False) |
                            df_unido['Descripcion'].str.contains('SALDO FINAL AL DIA', case=False, na=False)]

    # Ordenar las columnas en el DataFrame
    df_unido = df_unido[['Cuenta', 'Fecha', 'Descripcion', 'Referencia', 'Debitos', 'Creditos', 'Saldos']]

    # Leer la primera página para obtener el periodo del extracto
    periodo = tabula.read_pdf(ruta_pdf, pages=1, 
                              relative_area=True, 
                              relative_columns=True, 
                              area=[8, 0, 25, 100], 
                              columns=[35], silent=True, pandas_options={"header": None})

    # Procesar el nombre y el periodo del extracto
    denominacion = periodo[0][0][0].strip()
    desde_hasta = [celda for celda in list(periodo[0][1]) if "Extracto" in str(celda)][0].split(':')[-1].strip()

    # Dividir el texto en "desde" y "hasta"
    partes = desde_hasta.split(" al ")
    fecha_desde = partes[0]
    fecha_hasta = partes[1]

    # Reformatear las fechas al formato YYYY-MM-DD
    fecha_desde_reformateada = "-".join(reversed(fecha_desde.split("/")))
    fecha_hasta_reformateada = "-".join(reversed(fecha_hasta.split("/")))

    # Crear la nueva variable con el formato deseado
    desde_hasta_nuevo_formato = f"{fecha_desde_reformateada} al {fecha_hasta_reformateada}"

    # Crear el nombre del archivo Excel basado en la denominación y el periodo
    nombre_excel = f"{denominacion} Periodo del Extracto {desde_hasta_nuevo_formato}.xlsx"
    ruta_excel = os.path.join(carpeta_guardar, nombre_excel)

    # Crear el archivo Excel y guardar los datos
    with pd.ExcelWriter(ruta_excel, engine='xlsxwriter') as writer:
        lista_ctas = df_unido['Cuenta'].drop_duplicates()

        # Guardar cada cuenta en una hoja separada
        for cuenta in lista_ctas:
            df_cuenta = df_unido[df_unido['Cuenta'] == cuenta]
            df_cuenta.to_excel(writer, sheet_name=cuenta, index=False)
    
    print(f"Se ha guardado '{nombre_excel}' correctamente.")

In [ ]:
# Iterar sobre todos los archivos PDF en la carpeta
for archivo in os.listdir(carpeta_pdf):
    # Verificar si el archivo es un PDF
    if archivo.endswith(".pdf"):
        ruta_archivo = os.path.join(carpeta_pdf, archivo)
        try:
            # Procesar el archivo PDF
            procesar_pdf(ruta_archivo)
        except Exception as e:
            # Capturar y registrar el error con el nombre del archivo
            print(f"Error procesando el archivo '{archivo}': {e}")
